# XGboost with Hyperopt (Mean Absolute Error: 14902.082860659246)

Data from the [Housing Prices Competition for Kaggle Learn Users](https://www.kaggle.com/c/home-data-for-ml-course). 
Some of the codes are taken from the tutorial [Intermediate Machine Learning](https://www.kaggle.com/learn/intermediate-machine-learning)

### Reading the file

In [1]:
!python --version

Python 3.7.5


In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import GridSearchCV, cross_val_score
from hyperopt import hp, fmin, tpe, STATUS_OK, Trials
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
import time
from math import sqrt



In [3]:
# Read the data
X = pd.read_csv('train.csv', index_col='Id')
X_test_full = pd.read_csv('test.csv', index_col='Id')

In [4]:
print(X.shape)
print(X_test_full.shape)

(1460, 80)
(1459, 79)


In [5]:
display(X.head())

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,
1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [6]:
# Remove rows with missing target, separate target from predictors
X.dropna(axis=0, subset=['SalePrice'], inplace=True)

y = X.SalePrice              
X.drop(['SalePrice'], axis=1, inplace=True)

# Break off validation set from training data
X_train_full, X_valid_full, y_train, y_valid = train_test_split(X, y, test_size=0.3, random_state=42)

In [32]:
display(X.shape)
display(y.shape)

(1460, 79)

(1460,)

In [8]:
print(X_train_full.isnull().sum().sum())
print(X_valid_full.isnull().sum().sum())
print(y_train.isnull().sum().sum())
print(y_valid.isnull().sum().sum())
print(X_test_full.isnull().sum().sum())

4859
2106
0
0
7000


In [9]:
# "Cardinality" means the number of unique values in a column
# Select categorical columns with relatively low cardinality (convenient but arbitrary)
low_cardinality_cols = [cname for cname in X_train_full.columns if X_train_full[cname].nunique() < 10 and 
                        X_train_full[cname].dtype == "object"]

In [10]:
len(low_cardinality_cols)

40

In [11]:
# Select numeric columns
numeric_cols = [cname for cname in X_train_full.columns if X_train_full[cname].dtype in ['int64', 'float64']]

In [12]:
#total features retain 40 + 36 = 76
len(numeric_cols)

36

In [13]:
# Keep selected columns only
my_cols = low_cardinality_cols + numeric_cols
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()
X_test = X_test_full[my_cols].copy()
X_full = X[my_cols].copy() #for fitting full set later

print(X_train.shape)
print(X_valid.shape)
print(X_test.shape)
print(X_full.shape)

(1022, 76)
(438, 76)
(1459, 76)
(1460, 76)


In [14]:
# One-hot encode the data
X_train = pd.get_dummies(X_train)
X_valid = pd.get_dummies(X_valid)
X_test = pd.get_dummies(X_test)
X_full = pd.get_dummies(X_full) #for fitting full set later
X_train, X_valid = X_train.align(X_valid, join='left', axis=1)
X_train, X_test = X_train.align(X_test, join='left', axis=1)
X_train, X_full = X_train.align(X_full, join='left', axis=1, fill_value=0)

In [15]:
print(X_train.shape)
print(X_valid.shape)
print(X_test.shape)
print(X_full.shape)

(1022, 228)
(438, 228)
(1459, 228)
(1460, 228)


In [16]:
print(X_train.isnull().sum().sum())
print(X_valid.isnull().sum().sum())
print(X_test.isnull().sum().sum())
print(X_full.isnull().sum().sum())

247
10613
17838
348


In [17]:
# Define the model
my_model_1 = XGBRegressor(random_state=0)

# Fit the model
my_model_1.fit(X_train, y_train)
predictions_1 = my_model_1.predict(X_valid)

[23:27:09] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


/home/dylanpoh/anaconda3/envs/kagglehouse/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


In [18]:
# Get predictions
predictions_1 = my_model_1.predict(X_valid)

Finally, use the `mean_absolute_error()` function to calculate the mean absolute error (MAE) corresponding to the predictions for the validation set.  Recall that the labels for the validation data are stored in `y_valid`.

In [19]:
# Calculate MAE
mae_1 = mean_absolute_error(predictions_1,y_valid)
print("Mean Absolute Error:" , mae_1)

mse_1 = mean_squared_error(predictions_1, y_valid)
rootMeanSquaredError_1 = sqrt(mse_1)
print("RMSE:", rootMeanSquaredError_1)

Mean Absolute Error: 16233.512191424086
RMSE: 27448.393553403006


### Improving the model with Hyperopt

In [22]:
def objective(space):
    print(space)
    clf = XGBRegressor(n_estimators = space['n_estimators'],
                           max_depth = space['max_depth'],
                           min_child_weight = space['min_child_weight'],
                           subsample = space['subsample'],
                           learning_rate = space['learning_rate'],
                           gamma = space['gamma'],
                           colsample_bytree = space['colsample_bytree'],
                           objective='reg:linear',
                           random_state=42
                      )

    eval_set  = [(X_valid, y_valid)]

    clf.fit(X_train, y_train,
            eval_set=eval_set, eval_metric="mae",
            early_stopping_rounds=10,verbose=False)

    pred = clf.predict(X_valid)
    mae_scr = mean_absolute_error(y_valid, pred)
    print ("SCORE:", mae_scr)

    return {'loss':mae_scr, 'status': STATUS_OK }

space ={'max_depth': hp.choice('max_depth', np.arange(5, 30, dtype=int)),
        'min_child_weight': hp.quniform ('min_child', 1, 20, 1),
        'subsample': hp.uniform ('subsample', 0.8, 1),
        'n_estimators' : hp.choice('n_estimators', np.arange(1000, 10000, 100, dtype=int)),
        'learning_rate' : hp.quniform('learning_rate', 0.025, 0.5, 0.025),
        'gamma' : hp.quniform('gamma', 0.5, 1, 0.05),
        'colsample_bytree' : hp.quniform('colsample_bytree', 0.5, 1, 0.05)
    }

start_time = time.time()

trials = Trials()
best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=100,
            trials=trials)

elapsed_time = (time.time() - start_time)/60
print('Time taken to optimize the model: {} min.\n'.format(elapsed_time))

print (best)

{'colsample_bytree': 0.8, 'gamma': 0.55, 'learning_rate': 0.4, 'max_depth': 6, 'min_child_weight': 6.0, 'n_estimators': 1100, 'subsample': 0.9015998963014877}
[23:27:09] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
  0%|          | 0/100 [00:00<?, ?it/s, best loss: ?]

/home/dylanpoh/anaconda3/envs/kagglehouse/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \



SCORE:                                               
17727.205149472033                                   
{'colsample_bytree': 0.9500000000000001, 'gamma': 0.55, 'learning_rate': 0.30000000000000004, 'max_depth': 24, 'min_child_weight': 11.0, 'n_estimators': 3200, 'subsample': 0.9796674531502904}
[23:27:10] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
  1%|          | 1/100 [00:00<00:28,  3.42it/s, best loss: 17727.205149472033]

/home/dylanpoh/anaconda3/envs/kagglehouse/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \



SCORE:                                                                        
16937.201537528537                                                            
{'colsample_bytree': 0.7000000000000001, 'gamma': 0.9, 'learning_rate': 0.45, 'max_depth': 18, 'min_child_weight': 11.0, 'n_estimators': 5400, 'subsample': 0.8150743325609695}
[23:27:11] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
  2%|▏         | 2/100 [00:01<00:57,  1.70it/s, best loss: 16937.201537528537]

...

SCORE:                                                                         
16289.296277468608                                                             
{'colsample_bytree': 0.7000000000000001, 'gamma': 0.8, 'learning_rate': 0.2, 'max_depth': 27, 'min_child_weight': 10.0, 'n_estimators': 5400, 'subsample': 0.8640705141902739}
[23:29:49] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
 98%|█████████▊| 98/100 [02:39<00:03,  1.87s/it, best loss: 14961.378210616438]

/home/dylanpoh/anaconda3/envs/kagglehouse/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \



SCORE:                                                                         
16446.77877069064                                                              
{'colsample_bytree': 0.9, 'gamma': 0.75, 'learning_rate': 0.025, 'max_depth': 19, 'min_child_weight': 5.0, 'n_estimators': 7500, 'subsample': 0.8426706136136468}
[23:29:50] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
 99%|█████████▉| 99/100 [02:40<00:01,  1.59s/it, best loss: 14961.378210616438]

/home/dylanpoh/anaconda3/envs/kagglehouse/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \



SCORE:                                                                         
15518.149828767124                                                             
100%|██████████| 100/100 [02:48<00:00,  1.69s/it, best loss: 14961.378210616438]
Time taken to optimize the model: 2.814726543426514 min.

{'colsample_bytree': 0.6000000000000001, 'gamma': 0.8, 'learning_rate': 0.025, 'max_depth': 22, 'min_child': 5.0, 'n_estimators': 35, 'subsample': 0.8430136794230647}


In [23]:
print(best)

{'colsample_bytree': 0.6000000000000001, 'gamma': 0.8, 'learning_rate': 0.025, 'max_depth': 22, 'min_child': 5.0, 'n_estimators': 35, 'subsample': 0.8430136794230647}


**Take Note: The best from fmin always returns the index for parameters defined using hp.choice.**

See this issue which confirms this:

https://github.com/hyperopt/hyperopt/issues/216
To get the original values from best, use space_eval() function like this:

In [24]:
from hyperopt import space_eval
best_params = space_eval(space, best)
best_params

{'colsample_bytree': 0.6000000000000001,
 'gamma': 0.8,
 'learning_rate': 0.025,
 'max_depth': 27,
 'min_child_weight': 5.0,
 'n_estimators': 4500,
 'subsample': 0.8430136794230647}

In [25]:
hyperopt_bestmodel = XGBRegressor( **best_params)
hyperopt_bestmodel.fit(X_train,y_train)

[23:29:58] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


/home/dylanpoh/anaconda3/envs/kagglehouse/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.6000000000000001, gamma=0.8,
             importance_type='gain', learning_rate=0.025, max_delta_step=0,
             max_depth=27, min_child_weight=5.0, missing=None,
             n_estimators=4500, n_jobs=1, nthread=None, objective='reg:linear',
             random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
             seed=None, silent=None, subsample=0.8430136794230647, verbosity=1)

In [26]:
# Calculate MAE
predict_hyperopt = hyperopt_bestmodel.predict(X_valid)
mae_3 = mean_absolute_error(predict_hyperopt,y_valid)
print("Mean Absolute Error:" , mae_3)

#Calculate RMSE
mse_3 = mean_squared_error(predict_hyperopt, y_valid)
rootMeanSquaredError_3 = sqrt(mse_3)
print("RMSE:", rootMeanSquaredError_3)


Mean Absolute Error: 14902.082860659246
RMSE: 23269.87106022359


In [33]:
#Refit the model with full set of data
hyperopt_bestmodel.fit(X_full,y)

predict_hyperopt_full = hyperopt_bestmodel.predict(X_test)

/home/dylanpoh/anaconda3/envs/kagglehouse/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/home/dylanpoh/anaconda3/envs/kagglehouse/lib/python3.7/site-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


[23:44:16] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [34]:
predict_hyperopt_full.shape

(1459,)

In [36]:

# Save predictions in format used for competition scoring
output = pd.DataFrame({'Id': X_test.index,
                        'SalePrice': predict_hyperopt_full})
output.to_csv('submission.csv', index=False)

In [37]:
output.shape

(1459, 2)